In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import pickle
import time
import os
import shutil
import shap 

from torch.utils.data import DataLoader
from torch.nn import functional as F
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import LambdaLR, LinearLR, SequentialLR
from functools import partial
from sklearn.metrics import f1_score, accuracy_score
from sklearn.calibration import calibration_curve


from codes.models.metrics import calculate_roc_auc
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList, Patient
from codes.models.Transformers.dic_model_versions import DIC_MODEL_VERSIONS
from codes.tests.TestsClass import TestSet, TrainTransformerModel, TrainModel
import matplotlib.pyplot as plt



In [ ]:
##### Functionnal version with optionnal mask padding and dropouts, see Transformer_V1.ipynb for example
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)


### imports
import torch
import torch.nn as nn
import numpy as np
import time
from torch.nn import functional as F
from sklearn.metrics import f1_score, accuracy_score

from codes.models.Transformers.Embedding import EmbeddingPheno
from codes.models.metrics import calculate_roc_auc, calculate_classification_report, calculate_loss, get_proba

### Transformer's instance
# B, S, E, H, HN, MH = Batch_len, Sentence_len, Embedding_len, Head_size, Head number, MultiHead size.

def custom_softmax(input_tensor, dim=-1):
    # Calculer le softmax classique
    softmax_output = F.softmax(input_tensor, dim)

    # Trouver les colonnes avec que des -inf
    nan_columns = torch.all(input_tensor == float('-inf'), dim=dim)

    # Remplacer les valeurs de softmax par 0 pour les colonnes avec que des -inf
    softmax_output[nan_columns] = 0

    return softmax_output

class TransformerGeneModel_V2(nn.Module):
    def __init__(self, pheno_method, Embedding, Head_size, binary_classes, n_head, n_layer, mask_padding=False, padding_token=None, p_dropout=0, device='cpu', loss_version='cross_entropy', gamma=2, alpha=1, instance_size=None, proj_embed=True):
        super().__init__()
       
        self.Embedding_size = Embedding.Embedding_size
        self.binary_classes = binary_classes
        self.instance_size=instance_size
        self.proj_embed = proj_embed
        if not self.proj_embed:
            self.instance_size = self.Embedding_size
        if self.proj_embed:
            self.projection_embed = nn.Linear(self.Embedding_size, self.instance_size)
        self.classes_nb = 2 if self.binary_classes else 3
        self.blocks =PadMaskSequential(*[Block(self.instance_size, n_head=n_head, Head_size=Head_size, p_dropout=p_dropout) for _ in range(n_layer)]) #Block(self.instance_size, n_head=n_head, Head_size=Head_size) 
        self.ln_f = nn.LayerNorm(self.instance_size) # final layer norm
        self.lm_head_logits = nn.Linear(self.instance_size, self.classes_nb) 
        self.lm_head_proba = nn.Linear(self.instance_size,1) # plus one for the probabilities
        self.Embedding = Embedding
        self.mask_padding = mask_padding
        self.padding_token = padding_token
        self.padding_mask = None
        self.device = device
        self.pheno_method = pheno_method
        
        self.loss_version = loss_version
        self.gamma = gamma
        self.alpha = alpha
        self.shap = False
       
        self.diseases_embedding_table = Embedding.distinct_diseases_embeddings
        if self.pheno_method == 'Paul':
            self.counts_embedding_table = Embedding.counts_embeddings

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            

    def create_padding_mask(self, diseases_sentence):
        B, S = np.shape(diseases_sentence)[0], np.shape(diseases_sentence)[1]
        mask = torch.where(diseases_sentence==self.padding_token)
        padding_mask_mat = torch.ones((B, S, S), dtype=torch.int)
        padding_mask_mat[mask] = 0
        padding_mask_mat.transpose(-2,-1)[mask] = 0

        
        padding_mask_probas = torch.zeros((B, S))
        padding_mask_probas[mask] = -torch.inf
        padding_mask_probas = padding_mask_probas.view(B, S)
        return padding_mask_mat.to(self.device), padding_mask_probas.to(self.device) # 1 if masked, 0 else

    def set_padding_mask_transformer(self, padding_mask, padding_mask_probas):
        self.padding_mask = padding_mask
        self.padding_mask_probas = padding_mask_probas

    def forward(self, diseases_sentence, counts_diseases=None, targets=None):
        if counts_diseases == None:
            counts_diseases = torch.zeros(diseases_sentence.shape)

        diseases_sentence = diseases_sentence.to(torch.long)
        counts_diseases = counts_diseases.to(torch.long)

        Batch_len, Sentence_len = diseases_sentence.shape

        diseases_sentence = diseases_sentence.to(self.device)
        counts_diseases = counts_diseases.to(self.device)
        
        if targets!=None:
            targets = targets.to(self.device)

        if self.mask_padding:
            padding_mask, padding_mask_probas = self.create_padding_mask(diseases_sentence)
            self.set_padding_mask_transformer(padding_mask, padding_mask_probas)
            self.blocks.set_padding_mask_sequential(self.padding_mask)

        diseases_sentences_embedded = self.diseases_embedding_table(diseases_sentence) # shape B, S, E

        x = diseases_sentences_embedded 
    
        if self.pheno_method == 'Paul':
            counts_diseases_embedded = self.counts_embedding_table(counts_diseases) # shape B, S, E
            x = x + counts_diseases_embedded # shape B, S, E 
        
        if self.proj_embed:
            x = self.projection_embed(x)
        x = self.blocks(x) # shape B, S, E
        x = self.ln_f(x) # shape B, S, E
        logits = self.lm_head_logits(x) #shape B, S, Classes_Numb, token logits
        weights_logits = self.lm_head_proba(x).view(Batch_len, Sentence_len)
        if self.mask_padding:
            weights_logits = weights_logits + self.padding_mask_probas
        probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)
        #if self.mask_padding:
           # probas = probas * self.padding_mask_probas
        
        logits = (logits.transpose(1, 2) @ probas.view(Batch_len, Sentence_len, 1)).view(Batch_len, self.classes_nb)# (B,Classes_Numb) Weighted Average logits
        loss = calculate_loss(logits, targets, self.loss_version, self.gamma, self.alpha)

        if self.shap:
            return logits[:,0].view(Batch_len, 1)
        
        return logits, loss
    
    def forward_decomposed(self, diseases_sentence, diseases_count):
        self.list_attention_layers = []
        Batch_len, Sentence_len = diseases_sentence.shape
        print('coucou')
        diseases_sentence = diseases_sentence.to(self.device)
        counts_diseases = diseases_count.to(self.device)

        if self.mask_padding:
            padding_mask, padding_mask_probas = self.create_padding_mask(diseases_sentence)
            self.set_padding_mask_transformer(padding_mask, padding_mask_probas)
            self.blocks.set_padding_mask_sequential(self.padding_mask)

        diseases_sentences_embedded = self.diseases_embedding_table(diseases_sentence) # shape B, S, E

        x = diseases_sentences_embedded 
        if self.pheno_method == 'Paul':
            counts_diseases_embedded = self.counts_embedding_table(counts_diseases) # shape B, S, E
            x = x + counts_diseases_embedded # shape B, S, E 
        if self.proj_embed:
            x = self.projection_embed(x)
        x= self.blocks.forward_decompose(x, self.list_attention_layers) # shape B, S, E
        x_out = self.ln_f(x) # shape B, S, E
        logits = self.lm_head_logits(x_out) #shape B, S, Classes_Numb, token logits
        weights_logits = self.lm_head_proba(x).view(Batch_len, Sentence_len)
        if self.mask_padding:
            weights_logits = weights_logits + self.padding_mask_probas
        probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)
        #if self.mask_padding:
           # probas = probas * self.padding_mask_probas
        
        logits = (logits.transpose(1, 2) @ probas.view(Batch_len, Sentence_len, 1))# (B,Classes_Numb) Weighted Average logits
        return logits, probas, x_out
    

    def predict(self, diseases_sentence, diseases_count):
        logits, _ = self(diseases_sentence, diseases_count) # shape B, Classes_nb
        return torch.argmax(logits, dim=1)  # (B,)
        
    def predict_proba(self, diseases_sentence, diseases_count):
        logits, _ = self(diseases_sentence, diseases_count)
        probabilities = F.softmax(logits, dim=1)
        return probabilities
    
    def evaluate(self, dataloader_test):
        print('beginning inference evaluation')
        start_time_inference = time.time()
        predicted_labels_list = []
        predicted_probas_list = []
        true_labels_list = []

        total_loss = 0.
        self.eval()
        with torch.no_grad():
            for batch_sentences, batch_counts, batch_labels in dataloader_test:


                logits, loss = self(batch_sentences, batch_counts,  batch_labels)
                total_loss += loss.item()
                predicted_labels = self.predict(batch_sentences, batch_counts)
                predicted_labels_list.extend(predicted_labels.cpu().numpy())
                predicted_probas = self.predict_proba(batch_sentences, batch_counts)
                predicted_probas_list.extend(predicted_probas.cpu().numpy())
                true_labels_list.extend(batch_labels.cpu().numpy())
        f1 = f1_score(true_labels_list, predicted_labels_list, average='macro')
        accuracy = accuracy_score(true_labels_list, predicted_labels_list)
        auc_score = calculate_roc_auc(true_labels_list, np.array(predicted_probas_list)[:, 1], return_nan=True)
        proba_avg_zero, proba_avg_one = get_proba(true_labels_list, predicted_probas_list)
    
        self.train()
        print(f'end inference evaluation in {time.time() - start_time_inference}s')
        return f1, accuracy, auc_score, total_loss/len(dataloader_test), proba_avg_zero, proba_avg_one, predicted_probas_list, true_labels_list


    def write_embedding(self, writer):
        if self.proj_embed:
            embedding_tensor = self.projection_embed(self.diseases_embedding_table.weight).detach().cpu().numpy()
        else:
            embedding_tensor = self.diseases_embedding_table.weight.detach().cpu().numpy()

        writer.add_embedding(embedding_tensor, metadata=self.Embedding.metadata, metadata_header=["Name","Label"])


class PadMaskSequential(nn.Sequential):
    def __init__(self, *args):
        super(PadMaskSequential, self).__init__(*args)
        self.padding_mask = None

    def set_padding_mask_sequential(self, padding_mask):
        self.padding_mask = padding_mask

    def forward(self, x):
        for module in self:
            module.set_padding_mask_block(self.padding_mask)
            x = module(x)
        return x
    
    def forward_decompose(self, x, list_attention_layers):
        for module in self:
            module.set_padding_mask_block(self.padding_mask)
            x = module.forward_decompose(x, list_attention_layers)
        return x
    
class Block(nn.Module):
    def __init__(self, instance_size, n_head, Head_size, p_dropout):
        super().__init__()
        self.sa = MultiHeadSelfAttention(n_head, Head_size, instance_size, p_dropout)
        self.ffwd = FeedForward(instance_size, p_dropout)
        self.ln1 = nn.LayerNorm(instance_size)
        self.ln2 = nn.LayerNorm(instance_size)
        self.padding_mask = None

    def set_padding_mask_block(self, padding_mask):
        self.padding_mask = padding_mask

    def forward(self, x):
        self.sa.set_padding_mask_sa(self.padding_mask)
        #x = self.ln1(x)
        x = x + self.sa(x)
        x = self.ln1(x)
        x = x + self.ffwd(x)
        x = self.ln2(x)
        return x
    
    def forward_decompose(self, x, list_attention_layers=None):
        self.sa.set_padding_mask_sa(self.padding_mask)
        out_sa, attention_probas = self.sa.forward_decompose(x)
        if list_attention_layers != None:
            list_attention_layers.append(attention_probas)
        x = out_sa + x
        x = x + self.ffwd(x)
        x = self.ln2(x)
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, n_head, Head_size, instance_size, p_dropout):
        super().__init__()
        self.qkv_network = nn.Linear(instance_size, Head_size * 3, bias = False) #group the computing of the nn.Linear for q, k and v, shape 
        self.proj = nn.Linear(Head_size, instance_size)
        self.attention_dropout = nn.Dropout(p_dropout)
        self.resid_dropout = nn.Dropout(p_dropout)

        self.multi_head_size = Head_size // n_head
        self.flash = False
        self.n_head = n_head
        self.Head_size = Head_size
        self.padding_mask = None

    def set_padding_mask_sa(self, padding_mask):
        self.padding_mask = padding_mask

        #self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        # x of size (B, S, E)
        Batch_len, Sentence_len, _ = x.shape
        q, k, v  = self.qkv_network(x).split(self.Head_size, dim=2) #q, k, v of shape (B, S, H)
        # add a dimension to compute the different attention heads separately
        q_multi_head = q.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2) #shape B, HN, S, MH
        k_multi_head = k.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)
        v_multi_head = v.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)

        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            out = torch.nn.functional.scaled_dot_product_attention(q_multi_head, k_multi_head, v_multi_head, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:    
            attention_weights = (q_multi_head @ k_multi_head.transpose(-2, -1))/np.sqrt(self.multi_head_size) # shape B, S, S
            ### padding mask #####
            if self.padding_mask != None:
                padding_mask_weights = -(1-self.padding_mask)*(10**10)
                attention_weights = (attention_weights.transpose(0, 1)+padding_mask_weights).transpose(0, 1)
            #print(f'wei0={attention_weights}')
            attention_probas = F.softmax(attention_weights, dim=-1) # shape B, S, S
            if self.padding_mask != None:
                attention_probas = (attention_probas.transpose(0, 1)*self.padding_mask).transpose(0, 1)
           # attention_probas[attention_probas.isnan()]=0
            attention_probas = self.attention_dropout(attention_probas)

            #print(f'wei1={attention_probas}')
            #attention_probas = self.dropout(attention_probas)
            ## weighted aggregation of the values
            out = attention_probas @ v_multi_head # shape B, S, S @ B, S, MH = B, S, MH
        out = out.transpose(1, 2).contiguous().view(Batch_len, Sentence_len, self.Head_size)
        out = self.proj(out)
        out = self.resid_dropout(out)
        return out        
    
    def forward_decompose(self, x):
        # x of size (B, S, E)
        Batch_len, Sentence_len, _ = x.shape
        q, k, v  = self.qkv_network(x).split(self.Head_size, dim=2) #q, k, v of shape (B, S, H)
        # add a dimension to compute the different attention heads separately
        q_multi_head = q.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2) #shape B, HN, S, MH
        k_multi_head = k.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)
        v_multi_head = v.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)

        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            out = torch.nn.functional.scaled_dot_product_attention(q_multi_head, k_multi_head, v_multi_head, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:    
            attention_weights = (q_multi_head @ k_multi_head.transpose(-2, -1))/np.sqrt(self.multi_head_size) # shape B, S, S
            ### padding mask #####
            if self.padding_mask != None:
                padding_mask_weights = -(1-self.padding_mask)*(10**10)
                attention_weights = (attention_weights.transpose(0, 1)+padding_mask_weights).transpose(0, 1)


            attention_probas = F.softmax(attention_weights, dim=-1) # shape B, S, S
            if self.padding_mask != None:
                attention_probas = (attention_probas.transpose(0, 1)*self.padding_mask).transpose(0, 1)
           # attention_probas[attention_probas.isnan()]=0
            attention_probas_dropout = self.attention_dropout(attention_probas)


            #print(f'wei1={attention_probas}')
            #attention_probas = self.dropout(attention_probas)
            ## weighted aggregation of the values
            out = attention_probas_dropout @ v_multi_head # shape B, S, S @ B, S, MH = B, S, MH
        out = out.transpose(1, 2).contiguous().view(Batch_len, Sentence_len, self.Head_size)
        out = self.proj(out)
        #out = self.resid_dropout(out)
        return out, attention_probas   

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""
    def __init__(self, instance_size, p_dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear( instance_size, 4 * instance_size),
            nn.ReLU(),
            nn.Linear(4 * instance_size, instance_size),
            nn.Dropout(p_dropout)
        )

    def forward(self, x):
        return self.net(x)
        


In [ ]:
import os
import sys
sys.path.append('/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding')
import time
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class SineCosineEncoding(nn.Module):
    def __init__(self, Embedding_size, max_len=1000):
        super(SineCosineEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, Embedding_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, Embedding_size, 2).float() * -(np.log(10000.0) / Embedding_size))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)

    def forward(self, x):

        return self.encoding.to(x.device)[x]

class ZeroEmbedding(nn.Module):
    def __init__(self, Embedding_size, max_len=1000):
        super(ZeroEmbedding, self).__init__()
        self.encoding = torch.zeros(max_len, Embedding_size)
       

    def forward(self, x):

        return self.encoding.to(x.device)[x]


class EmbeddingPheno(nn.Module):
    def __init__(self, method=None, counts_method=None, vocab_size=None, max_count_same_disease=None, Embedding_size=None, rollup_depth=4, freeze_embed=False, dicts=None):
        super(EmbeddingPheno, self).__init__()

        self.dicts = dicts
        self.rollup_depth = rollup_depth
        self.nb_distinct_diseases_patient = vocab_size
        self.Embedding_size = Embedding_size
        self.max_count_same_disease = None
        self.metadata = None
        self.counts_method = counts_method

        if self.dicts != None:
            id_dict = self.dicts['id']
            name_dict = self.dicts['name']
            cat_dict = self.dicts['cat']
            codes = list(id_dict.keys())
            diseases_present = self.dicts['diseases_present']
            self.metadata = [[name_dict[code], cat_dict[code]] for code in codes]

        
        if method == None:
            self.distinct_diseases_embeddings = nn.Embedding(vocab_size, Embedding_size)
            self.counts_embeddings = nn.Embedding(max_count_same_disease, Embedding_size)
            torch.nn.init.normal_(self.distinct_diseases_embeddings.weight, mean=0.0, std=0.02)
            torch.nn.init.normal_(self.counts_embeddings.weight, mean=0.0, std=0.02)

        elif method == 'Abby':
            embedding_file_diseases = f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
            pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
            self.Embedding_size = pretrained_weights_diseases.shape[1]

            self.distinct_diseases_embeddings = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed)
            self.counts_embeddings = nn.Embedding(max_count_same_disease, self.Embedding_size)



        elif method=='Paul':
            embedding_file_diseases = f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Embeddings/Paul_Glove/glove_UKBB_omop_rollup_closest_depth_{self.rollup_depth}_no_1_diseases.pth'
            pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
            self.Embedding_size = pretrained_weights_diseases.shape[1]

            self.distinct_diseases_embeddings = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed)
            if self.counts_method == 'SineCosine':
                self.counts_embeddings = SineCosineEncoding(self.Embedding_size, max_count_same_disease)
            elif self.counts_method == 'no_counts':
                self.counts_embeddings = ZeroEmbedding(self.Embedding_size, max_count_same_disease )
            else:

                self.counts_embeddings = nn.Embedding(max_count_same_disease, self.Embedding_size)
    def write_embedding(self, writer):
            embedding_tensor = self.distinct_diseases_embeddings.weight.data.detach().cpu().numpy()
            writer.add_embedding(embedding_tensor, metadata=self.metadata, metadata_header=["Name","Label"])


class EmbeddingPhenoCat(nn.Module):
    def __init__(self, pheno_method=None,  method=None, proj_embed=None, counts_method=None, Embedding_size=10, instance_size=10, rollup_depth=4, freeze_embed=False, dic_embedding_cat_params={}, dicts=None, device='cpu'):
        super(EmbeddingPhenoCat, self).__init__()

        self.rollup_depth = rollup_depth
        self.Embedding_size = Embedding_size
        self.max_count_same_disease = None
        self.dic_embedding_cat_params = dic_embedding_cat_params
        dic_embedding_cat = {}
        self.method = method
        self.pheno_method = pheno_method
        self.dicts = dicts
        self.proj_embed = proj_embed
        self.projection_embed = None
        self.instance_size = instance_size
        self.counts_method = counts_method

        self.device = device
        if self.dicts != None:
            id_dict = self.dicts['id']
            name_dict = self.dicts['name']
            cat_dict = self.dicts['cat']
            codes = list(id_dict.keys())
            diseases_present = self.dicts['diseases_present']
            self.metadata = [[name_dict[code], cat_dict[code]] for code in codes]

        for cat, max_number  in self.dic_embedding_cat_params.items():
        
            if cat=='diseases':
                if self.method == None:
                    dic_embedding_cat[cat] = nn.Embedding(max_number, Embedding_size)
                    torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

                elif self.method == 'Abby':
                    embedding_file_diseases = f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
                    pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
                    self.Embedding_size = pretrained_weights_diseases.shape[1]
                    dic_embedding_cat[cat] = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed).to(self.device)

            

                elif self.method=='Paul':
                    embedding_file_diseases = f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Embeddings/Paul_Glove/glove_UKBB_omop_rollup_closest_depth_{self.rollup_depth}_no_1_diseases.pth'
                    pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
                    self.Embedding_size = pretrained_weights_diseases.shape[1]
                    dic_embedding_cat[cat] = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed).to(self.device)
                    
            elif cat == 'counts':
                if self.pheno_method == 'Paul':
                    if self.counts_method[cat] == 'SineCosine':
                        dic_embedding_cat[cat] = SineCosineEncoding(self.instance_size, max_number).to(self.device)
                    elif self.counts_method[cat] == 'no_counts':
                        dic_embedding_cat[cat] = ZeroEmbedding(self.instance_size, max_number).to(self.device)
                    else:
                        dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                        torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

            elif cat == 'age':
                if self.counts_method[cat] == 'SineCosine':
                    dic_embedding_cat[cat] = SineCosineEncoding(self.instance_size, max_number).to(self.device)
                elif self.counts_method[cat] == 'no_counts':
                    dic_embedding_cat[cat] = ZeroEmbedding(self.instance_size, max_number).to(self.device)
                else:
                    dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                    torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

                    

            else:
                dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

        if self.proj_embed:
            self.projection_embed = nn.Linear(self.Embedding_size, self.instance_size).to(self.device)

        self.dic_embedding_cat = dic_embedding_cat

    def forward(self, input_dict):
        list_env_embedded = []
        for key, value in input_dict.items():
            
            batch_len = len(value)

            if key=='diseases':
                diseases_sentences_embedded = self.dic_embedding_cat[key](value)
                if self.proj_embed:
                    diseases_sentences_embedded = self.projection_embed(diseases_sentences_embedded)

            elif key=='counts':
                if self.pheno_method == 'Paul':
                    counts_sentence_embedded = self.dic_embedding_cat[key](value)
                    diseases_sentences_embedded = diseases_sentences_embedded + counts_sentence_embedded
            

            else:
                list_env_embedded.append(self.dic_embedding_cat[key](value).view(batch_len, 1, self.instance_size))

        env_embedded = torch.concat(list_env_embedded, dim=1)

        return torch.concat([diseases_sentences_embedded, env_embedded], dim=1)
            



In [ ]:
train_model =  TrainModel.load_instance_test(121)
model = train_model.model
model.device = 'cpu'
patient_list = train_model.patient_list
data = train_model.dataT
nb_max_distinct_disease = len(patient_list[0].diseases_sentence)

indices_train, indices_test = train_model.dataT.indices_train, train_model.dataT.indices_test
train_model.patient_list_transformer_train, train_model.patient_list_transformer_test = train_model.patient_list.get_transformer_data(indices_train.astype(int), indices_test.astype(int))
#creation of torch Datasets:
dataloader_train = DataLoader(train_model.patient_list_transformer_train, batch_size=train_model.batch_size, shuffle=True)
dataloader_test = DataLoader(train_model.patient_list_transformer_test, batch_size=train_model.batch_size, shuffle=True)



In [ ]:
batch_sentence, batch_counts, batch_labels = next(iter(dataloader_test))


In [ ]:
Embedding = EmbeddingPheno(method=train_model.embedding_method, counts_method=train_model.counts_method, vocab_size=train_model.vocab_size, 
                           max_count_same_disease=train_model.max_count_same_disease, Embedding_size=train_model.Embedding_size, rollup_depth=4, freeze_embed=train_model.freeze_embedding, 
                           dicts=train_model.dataT.dicts)

In [ ]:


model = TransformerGeneModel_V2(pheno_method=train_model.pheno_method,
                                Embedding=Embedding,
                                Head_size=train_model.Head_size,
                                binary_classes=True,
                                n_head=train_model.n_head,
                                n_layer=2,
                                instance_size=train_model.instance_size,
                                mask_padding=True,
                                padding_token=0,
                                device='cpu')

In [ ]:
model.shap=True

In [ ]:
input_data = batch_sentence.to(torch.float)
shap_input = batch_sentence.to(torch.float)[0].view(1, 122)

In [ ]:
explainer = shap.DeepExplainer(model, input_data)
shap_values = explainer.shap_values(input_data)


In [ ]:
input_data = [batch_sentence.to(torch.float), batch_counts.to(torch.float)]
shap_input = [batch_sentence.to(torch.float)[0].view(1, 122), batch_counts.to(torch.float)[0].view(1, 122)]

In [ ]:
input_data[1]

In [ ]:
model(*shap_input)

In [ ]:
explainer = shap.DeepExplainer(model, input_data)

In [ ]:
shap_values = explainer.shap_values(shap_input)


In [ ]:
len(shap_values[0][0])


In [ ]:
shap_values[0]

In [ ]:
shap.summary_plot(shap_values[0], features=shap_input[0])
